In [1]:
import jieba
import pandas as pd

df = pd.read_csv('data/消费content.csv', encoding='utf-8')  # 打开文件


# print(df.head())  # 默认读取文件的头5行
# print(df.shape)   # 查看数据的维度

# 使用jieba分词对文件进行分词
def chinese_word_cut(text):
    return " ".join(jieba.cut(text))
# 设置pd的显示长度
pd.set_option('max_colwidth', 500)

df["content_cutted"] = df.content.apply(chinese_word_cut)
print(df.content_cutted.head()) # 查看文件是否已被正确分词

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\李大帅\AppData\Local\Temp\jieba.cache
Loading model cost 0.723 seconds.
Prefix dict has been built successfully.


0                                                                                                                                                                \n                                         【 海南 # 三亚 100 吨 冬季 瓜菜 驰援 武汉 # 】 1 日 ， 海南 冬季 瓜菜 驰援 武汉 活动 仪式 在 三亚 启动 。 此批 供应 武汉 冬季 蔬菜 共 100 吨 ， 主要 有 豇豆 、 青瓜 、 苦瓜 、 葫芦 瓜 、 茄子 、 辣椒 等 武汉市 民 认可度 较 高且 消费量 较大 的 瓜菜 ， 另外 还有 9 吨 福返 “ 树上 熟 ” 芒果 。 这是 海南 无偿 捐送 武汉 的 首批 爱心 瓜菜 和 水果 。 据悉 ， 约 27 个 小时 后 ， 这批 新鲜 瓜菜 将 运抵 湖北省 ， 交接 给 武汉市 商务局 。 （ 黄晓慧   沙晓峰 ）   收起 全文 d \n
1                                                                                                                                                                     \n                                         【 海南 # 三亚 100 吨 冬季 瓜菜 驰援 武汉 # 】 1 日 ， 海南 冬季 瓜菜 驰援 武汉 活动 仪式 在 三亚 启动 。 此批 供应 武汉 冬季 蔬菜 共 100 吨 ， 主要 有 豇豆 、 青瓜 、 苦瓜 、 葫芦 瓜 、 茄子 、 辣椒 等 武汉市 民 认可度 较 高且 消费量 较大 的 瓜菜 ， 另外 还有 9 吨 福返 “ 树上 熟 ” 芒果 。 这是 海南 无偿 捐送 武汉 的 首批 爱心 瓜菜 和 水果 。 据悉 ， 约 27 个 小时 后 ， 这批 新鲜 瓜菜 将 运抵 湖北省 ， 交接 给 武汉市 商务局 。 （ 人民日报 ）   收

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

'''
    文本向量化：将文章中的关键词转换为一个个特征，然后统计关键词个数
'''
n_features = 1000  # 只提取1000个最重要的关键词
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df=0.5,
                                min_df=10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

from sklearn.decomposition import LatentDirichletAllocation

n_topics = 5  # 设定主题个数
lda = LatentDirichletAllocation(max_iter=50, learning_method='online', learning_offset=50.,
                                random_state=0)
result_topic = lda.fit(tf)
print(result_topic)

In [ ]:
# 显示每个主题下的前若干关键词
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print()


n_top_words = 20
tf_features_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_features_names, n_top_words)

import pyLDAvis.sklearn

# 主题可视化
vis = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.show(vis)